In [1]:
# https://tarunlalwani.com/post/reusing-existing-browser-session-selenium/
# https://medium.com/better-programming/lets-create-an-instagram-bot-to-show-you-the-power-of-selenium-349d7a6744f7
# https://towardsdatascience.com/web-scraping-using-selenium-and-beautifulsoup-99195cd70a58
# https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058
import time
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests

%matplotlib inline
import matplotlib.pyplot as plt
import uuid
import pandas as pd

## Open browser to login page

In [2]:
email = 'harvestharvest894@gmail.com'
password = 'earplugs99'

browser = webdriver.Chrome()
browser.get('https://www.instagram.com/accounts/login/ ')

## Login

In [3]:
emailInput = browser.find_elements_by_css_selector('form input')[0]
passwordInput = browser.find_elements_by_css_selector('form input')[1]

emailInput.send_keys(email)
passwordInput.send_keys(password)
passwordInput.send_keys(Keys.ENTER)

time.sleep(2)

## Dismiss notification pop

In [4]:
browser.find_element_by_xpath('//button[text()="Not Now"]').click()

## Search for profile

In [5]:
# search_input = browser.find_element_by_xpath("//input[@placeholder='Search']")

In [6]:
# search_input.send_keys('pizza')
# time.sleep(1)

## Scroll through page and collect source

In [89]:
more_button = browser.find_element_by_xpath('//button[text()="more"]')

In [90]:
more_button.click()

In [74]:
for more_button in browser.find_elements_by_xpath('//button[text()="more"]'):
    more_button.click()

ElementClickInterceptedException: Message: element click intercepted: Element <button class="sXUSN">...</button> is not clickable at point (64, 9). Other element would receive the click: <div class="                  Igw0E   rBNOH        eGOV_     ybXk5    _4EzTm                                                                                                              ">...</div>
  (Session info: chrome=78.0.3904.97)


In [56]:
browser.get_window_size()['height']

900

In [72]:
browser.execute_script("window.scrollBy(0, {});".format(browser.get_window_size()['height']))

In [73]:
SCROLL_PAUSE_TIME = 1.0
soups = []
# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")
# last_height = browser.get_window_size()['height']

for _ in range(100):
    for more_button in browser.find_elements_by_xpath('//button[text()="more"]'):
        print('Clicking more')
        more_button.click()
    time.sleep(SCROLL_PAUSE_TIME)
    
    soups.append(BeautifulSoup(browser.page_source))
    
    # Scroll down to bottom
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    browser.execute_script("window.scrollTo(0, {});".format(browser.get_window_size()['height']))

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    print('New height: {}'.format(new_height))
    print('New height: {}'.format(last_height))
    if new_height == last_height:
        break
    
    last_height = new_height
    break

Clicking more


ElementClickInterceptedException: Message: element click intercepted: Element <button class="sXUSN">...</button> is not clickable at point (86, 9). Other element would receive the click: <div class="                  Igw0E   rBNOH        eGOV_     ybXk5    _4EzTm                                                                                                              ">...</div>
  (Session info: chrome=78.0.3904.97)


## Grab all posts

In [8]:
# soup = BeautifulSoup(browser.page_source)

# articles = soup.find_all('article')

In [9]:
def extract_from_post_article(article: bs4.element.Tag):
    post_images = [image for image in article.find_all('img') if 'profile picture' not in image.get('alt')]
    post_image = post_images[0]
    alt_text = post_image.get('alt')
    img_insta_url = post_image.get('src')
    
    influencer = article.contents[0].text
    
    caption = article.contents[2].find_all(recursive=False)[2].text
    if caption.startswith(influencer):
        caption = caption[len(influencer):]

    caption = caption.split('View all')[0]
    
    id = uuid.uuid4()
    
    r = requests.get(img_insta_url)
    img_file = "{}_{}.jpg".format(influencer, id)
    with open(img_file, 'wb') as f:
        f.write(r.content)
        
    return {
        'alt_text': alt_text,
        'img_insta_url': img_insta_url,
        'influencer': influencer,
        'caption': caption,
        'id': id,
        'img_file': img_file,
    }
    
    
# extract_from_post_article(articles[0])

In [10]:
def process_soup(soup):
    articles = soup.find_all('article')
    
    processed_articles = []
    for article in articles:
        processed_articles.append(
            extract_from_post_article(article)
        )
    return pd.DataFrame(processed_articles)

article_dfs = [process_soup(soup) for soup in soups]

In [11]:
scrapped_df = pd.concat(article_dfs)
scrapped_df.head()

,alt_text,img_insta_url,influencer,caption,id,img_file
0,Image may contain: dog,https://scontent-yyz1-1.cdninstagram.com/vp/a3...,puppytoday,Caption this photo.,629ad639-be9e-4ffe-8e9b-36d001ef97a2,puppytoday_629ad639-be9e-4ffe-8e9b-36d001ef97a...
1,Image may contain: dog and text,https://scontent-yyz1-1.cdninstagram.com/vp/ee...,dogs.therapy,❤️... more,1a9ce979-ea5c-40e7-a2d8-21f683f324e9,dogs.therapy_1a9ce979-ea5c-40e7-a2d8-21f683f32...
2,No photo description available.,https://scontent-yyz1-1.cdninstagram.com/vp/e6...,puppytoday,Caption this photo,524876a4-c434-4adf-93ec-3038e30898eb,puppytoday_524876a4-c434-4adf-93ec-3038e30898e...
0,,https://scontent-yyz1-1.cdninstagram.com/vp/36...,dogs.therapy,Oh Hello.👋🏻📽... more,d4926f20-a140-4e1a-b1f1-1d3779801696,dogs.therapy_d4926f20-a140-4e1a-b1f1-1d3779801...
1,Image may contain: outdoor and nature,https://scontent-yyz1-1.cdninstagram.com/vp/d2...,puppytoday,,3f9e810e-5bd5-46b9-9b87-ba62c3dc0b9a,puppytoday_3f9e810e-5bd5-46b9-9b87-ba62c3dc0b9...


In [35]:
scrapped_df['id'] = (scrapped_df['influencer'] + scrapped_df['caption']).apply(lambda x: str(hash(x)))

In [13]:

from application.models import InfluencerPost

In [23]:
from contextlib import contextmanager
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

DATABASE_URL="postgresql://localhost:5432/insta_clone"
engine = db.create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [40]:
def save_post_if_new(post, session):
    print('Id: {}'.format(scrapped['id']))
    existing_matches = session.query(InfluencerPost).filter_by(id=post['id']).first()
    if not existing_matches:
        print('No existing post found, saving')
        influencer_post = InfluencerPost(**post[['id', 'influencer', 'img_insta_url', 'caption', 'alt_text']].to_dict())
        session.add(influencer_post)
        session.commit()
        
with session_scope() as session:
    scrapped_df.apply(save_post_if_new, axis=1, session=session)

Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
Id: -4983744667383358256
